In [1]:
import pennylane as qml
import numpy as np
from itertools import combinations

In [2]:
def find_n(feature:list):
    
    max_side = np.max(feature)
    binary = bin(max_side)[2:]
    n = len(binary)

    return n

In [70]:
def is_rectangle(sides:list):
    n_qubits = find_n(sides)

    measurement_qubits = range(3)
    n_measure = len(measurement_qubits)

    n_qubits_circ = n_measure+4*n_qubits

    dev_basis = qml.device('default.qubit',wires=n_qubits_circ+1,shots=100)

    wires_a = range(n_measure,n_qubits+n_measure)
    wires_b = range(n_qubits+n_measure,2*n_qubits+n_measure)
    wires_c = range(2*n_qubits+n_measure,3*n_qubits+n_measure)
    wires_d = range(3*n_qubits+n_measure,4*n_qubits+n_measure)
    wires = [wires_a,wires_b,wires_c,wires_d]
    m_result = []
    def swap(wires_0,wires_1,control:int):
        #Comparing a and b - or rather checking if a == b - returns |0> on the ancilla if equal and returns |+>
        for i,j in zip(wires_0,wires_1):
            qml.ctrl(qml.SWAP([i,j]),control=control)


    @qml.qnode(dev_basis)
    def base_enc(feature:list):
    
        qml.BasisEmbedding(features=feature[0], wires=wires_a)
        qml.BasisEmbedding(features=feature[1], wires=wires_b)
        qml.BasisEmbedding(features=feature[2], wires=wires_c)
        qml.BasisEmbedding(features=feature[3], wires=wires_d)
        
    
        qml.Hadamard(measurement_qubits[0])
        swap(wires_a,wires_b,measurement_qubits[0])
        swap(wires_c,wires_d,measurement_qubits[0])
        qml.Hadamard(measurement_qubits[0])

        qml.Hadamard(measurement_qubits[1])
        swap(wires_a,wires_c,measurement_qubits[1])
        swap(wires_b,wires_d,measurement_qubits[1])
        qml.Hadamard(measurement_qubits[1])

        qml.Hadamard(measurement_qubits[2])
        swap(wires_a,wires_d,measurement_qubits[2])
        swap(wires_b,wires_c,measurement_qubits[2])
        qml.Hadamard(measurement_qubits[2])

        return qml.probs(measurement_qubits[0]),qml.probs(measurement_qubits[1]),qml.probs(measurement_qubits[2])
    
    circ_result = base_enc(sides)
    rectangle = 0
    for i,j in enumerate(circ_result):
        if  j[0]==1:
            rectangle = rectangle + 1
            
    if rectangle == 3 or rectangle == 1:
        return 1

    return 0


is_rectangle([21,2,2,21])

1